In [ ]:
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import random
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
def build_rnn(input_shape: tuple, num_classes: int) -> nn.Module:
    C, H, W = input_shape
    input_size = C * W
    hidden_size = random.choice([64, 128, 256])
    num_layers = random.randint(1, 3)
    bidirectional = random.choice([False, True])
    cell_type = random.choice(['LSTM', 'GRU'])

    class RNNClassifier(nn.Module):
        def __init__(self):
            super().__init__()
            Cell = nn.LSTM if cell_type == 'LSTM' else nn.GRU
            self.rnn = Cell(input_size, hidden_size, num_layers,
                            batch_first=True, bidirectional=bidirectional)
            self.fc = nn.Linear(hidden_size * (2 if bidirectional else 1),
                                num_classes)

        def forward(self, x):
            B = x.size(0)
            seq = x.view(B, C, H, W).permute(0, 2, 1, 3).contiguous().view(B, H, C * W)
            out, _ = self.rnn(seq)
            last = out[:, -1, :]
            return self.fc(last)

    return RNNClassifier()

In [ ]:
def extract_probe_features(model, X, y, criterion):
    model.to(DEVICE).train()
    logP = np.log(sum(p.numel() for p in model.parameters()))
    logB = np.log(min(32, X.size(0)))
    Xp, yp = X[:32].to(DEVICE), y[:32].to(DEVICE)
    params = [p for p in model.parameters() if p.requires_grad]
    g2_list, tau_list = [], []
    for xi, yi in zip(Xp, yp):
        xi, yi = xi.unsqueeze(0), yi.unsqueeze(0)
        model.zero_grad()
        logits = model(xi)
        loss = criterion(logits, yi)
        grads = torch.autograd.grad(loss, params, retain_graph=True)
        gv = torch.cat([g.contiguous().view(-1) for g in grads])
        g2_list.append((gv**2).sum().item())
        model.zero_grad()
        true_logit = logits.view(-1)[yi.item()]
        grads_f = torch.autograd.grad(true_logit, params, retain_graph=True)
        fv = torch.cat([g.contiguous().view(-1) for g in grads_f])
        tau_list.append((fv**2).sum().item())
    logG2 = np.log(np.mean(g2_list))
    logTau = np.log(np.sum(tau_list))
    return np.array([logP, logB, logG2, logTau])

In [ ]:
def measure_convergence(model, X, y, eps, lr, criterion):
    model.to(DEVICE).train()
    X, y = X.to(DEVICE), y.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    init_loss = None
    for t in range(1, 501):
        optimizer.zero_grad()
        logits = model(X)
        loss = criterion(logits, y)
        if t == 1:
            init_loss = loss.item()
        if loss.item() <= eps * init_loss:
            return t
        loss.backward()
        optimizer.step()
    return 500

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom RNN datasets
class PermutedMNIST(datasets.MNIST):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        seed = 42
        torch.manual_seed(seed)
        self.permutation = torch.randperm(28 * 28)

    def __getitem__(self, index):
        img, target = super().__getitem__(index)
        img = img.view(-1)[self.permutation].view(1, 28, 28)
        return img, target

class SequentialMNIST(datasets.MNIST):
    def __getitem__(self, index):
        img, target = super().__getitem__(index)
        img = img.view(1, 28, 28)
        return img, target

DATASETS = {
    'MNIST': datasets.MNIST,
    'FashionMNIST': datasets.FashionMNIST,
    'PermutedMNIST': PermutedMNIST,
    'SequentialMNIST': SequentialMNIST
}
TRANSFORMS = {name: transforms.Compose([transforms.ToTensor()]) for name in DATASETS}

LR_VALUES = [0.0005, 0.001, 0.005]
BATCH_SIZES = [50, 100]
EPS_VALUES = [0.1, 0.15, 0.2]
N_EVAL_TRIALS = 10

In [ ]:
df_meta = pd.read_csv('../meta_datasets/meta_dataset_rnn.csv')
FEATURES = ['logP', 'logB', 'logG2', 'logTau', 'logLR', 'logN']
X_meta = df_meta[FEATURES].values
y_meta = df_meta['T_star'].values
meta_reg = XGBRegressor(n_estimators=200, max_depth=4, random_state=42)
meta_reg.fit(X_meta, y_meta)

In [ ]:
print("CAPE RNN Evaluation:")
records = []

for ds_name, ds_cls in DATASETS.items():
    ds = ds_cls(root='./data', train=True, download=True, transform=TRANSFORMS[ds_name])
    num_classes = len(ds.classes)
    input_shape = ds[0][0].shape
    total_N = len(ds)
    criterion = nn.CrossEntropyLoss()

    for lr in LR_VALUES:
        logLR = np.log(lr)
        for B in BATCH_SIZES:
            loader = DataLoader(ds, batch_size=B, shuffle=True)
            for eps in EPS_VALUES:
                y_preds, y_trues = [], []
                for _ in range(N_EVAL_TRIALS):
                    model = build_rnn(input_shape, num_classes)
                    Xp, yp = next(iter(loader))
                    z0 = extract_probe_features(model, Xp, yp, criterion)
                    z = np.concatenate([z0, [logLR, np.log(total_N)]])
                    T_pred = meta_reg.predict(z.reshape(1, -1))[0]
                    T_act = measure_convergence(model, Xp, yp, eps, lr, criterion)
                    y_preds.append(T_pred)
                    y_trues.append(T_act)

                mae = mean_absolute_error(y_trues, y_preds)
                corr = np.corrcoef(y_trues, y_preds)[0, 1]
                print(f"{ds_name} | lr={lr} | B={B} | eps={eps} | TT error={mae:.0f} | Corr={corr:.2f}")
                records.append({
                    'dataset': ds_name,
                    'TT_error': mae,
                    'Corr': corr
                })

In [ ]:
df_results = pd.DataFrame(records)
df_summary = df_results.groupby("dataset").agg({
    "TT_error": "mean",
    "Corr": "mean"
}).reset_index()

df_summary["TT_error"] = df_summary["TT_error"].round(2)
df_summary["Corr"] = df_summary["Corr"].round(3)

# --- Save results ---
df_summary.to_csv("RNN_evaluation_dataset_avg.csv", index=False)
print("Per-dataset average results saved to RNN_evaluation_dataset_avg.csv")